# Avfallspolisen – Notebook
Analyserar bilder och videoströmmar för att kontrollera om avfall är korrekt sorterat
med hjälp av YOLOv8 och OpenCV.

**Tre klasser:**
- Klass 0: Dryckeskartong (mjölk, yoghurt, etc.)
- Klass 1: Konservburk
- Klass 2: Pantburk

## 1. Installation & Imports

In [ ]:
# Installera nödvändiga paket (kör vid behov)
# !pip install ultralytics opencv-python numpy matplotlib

In [ ]:
import warnings
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

print('Alla paket importerade!')

## 2. Konfiguration

In [ ]:
# Avståndströskel i pixlar – objekt inom detta avstånd anses vara "nära"
PROXIMITY_THRESHOLD = 150

# Klassnamn: singular för enskilda objekt, plural för sorterade grupper
CLASS_NAMES = {
    0: {"singular": "Dryckeskartong", "plural": "KARTONGER"},
    1: {"singular": "Konservburk",    "plural": "KONSERVBURKAR"},
    2: {"singular": "Pantburk",       "plural": "PANTBURKAR"},
}

# Färger i BGR-format (OpenCV) och RGB-format (matplotlib)
COLOR_SORTED   = (0, 255, 0)    # Grön  – korrekt sorterat
COLOR_UNSORTED = (0, 0, 255)    # Röd   – felosorterat
COLOR_ALONE    = (0, 255, 255)  # Gul   – ensamt objekt

print('Konfiguration laddad.')

## 3. Hjälpfunktioner

In [ ]:
def berakna_centroid(box):
    """Beräknar centroiden (mittpunkten) för en bounding box [x1, y1, x2, y2]."""
    x1, y1, x2, y2 = box
    return int((x1 + x2) / 2), int((y1 + y2) / 2)


def berakna_avstand(punkt1, punkt2):
    """Beräknar det euklidiska avståndet mellan två punkter (x, y)."""
    return float(np.sqrt((punkt1[0] - punkt2[0])**2 + (punkt1[1] - punkt2[1])**2))


class UnionFind:
    """Union-Find-struktur för att gruppera nära objekt."""

    def __init__(self, n):
        self.parent = list(range(n))

    def hitta(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.hitta(self.parent[x])
        return self.parent[x]

    def forena(self, x, y):
        rx, ry = self.hitta(x), self.hitta(y)
        if rx != ry:
            self.parent[rx] = ry


def bestam_status(detektioner):
    """
    Bestämmer sorteringsstatus för alla detekterade objekt med Union-Find.

    Returnerar en lista med dicts: {box, class_id, conf, status, grupp_id, centroid}.
    Status: 'sorted' | 'unsorted' | 'alone'
    """
    n = len(detektioner)
    if n == 0:
        return []

    centroider = [berakna_centroid(d['box']) for d in detektioner]
    uf = UnionFind(n)

    for i in range(n):
        for j in range(i + 1, n):
            if berakna_avstand(centroider[i], centroider[j]) <= PROXIMITY_THRESHOLD:
                uf.forena(i, j)

    grupper = {}
    for i in range(n):
        rot = uf.hitta(i)
        grupper.setdefault(rot, []).append(i)

    resultat = []
    for i, det in enumerate(detektioner):
        rot = uf.hitta(i)
        grupp_index = grupper[rot]
        klasser_i_grupp = {detektioner[j]['class_id'] for j in grupp_index}

        if len(grupp_index) == 1:
            status = 'alone'
        elif len(klasser_i_grupp) == 1:
            status = 'sorted'
        else:
            status = 'unsorted'

        resultat.append({**det, 'status': status, 'grupp_id': rot, 'centroid': centroider[i]})

    return resultat


def rita_pa_frame(frame, resultat):
    """Ritar bounding boxes och text på framen baserat på sorteringsstatus."""
    sorterade_grupper = {}

    for det in resultat:
        status = det['status']
        box = det['box']
        x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
        klassinfo = CLASS_NAMES.get(det['class_id'], {'singular': '?', 'plural': '?'})

        if status == 'sorted':
            cv2.rectangle(frame, (x1, y1), (x2, y2), COLOR_SORTED, 2)
            sorterade_grupper.setdefault(det['grupp_id'], []).append(det)
        elif status == 'unsorted':
            cv2.rectangle(frame, (x1, y1), (x2, y2), COLOR_UNSORTED, 2)
            cv2.putText(frame, klassinfo['singular'], (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_UNSORTED, 2)
        else:  # alone
            cv2.rectangle(frame, (x1, y1), (x2, y2), COLOR_ALONE, 2)
            cv2.putText(frame, klassinfo['singular'], (x1, y1 - 8),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_ALONE, 2)

    # Rita grupptext EN gång per sorterad grupp
    for grupp_id, dets in sorterade_grupper.items():
        klassinfo = CLASS_NAMES.get(dets[0]['class_id'], {'singular': '?', 'plural': '?'})
        grupp_x1 = min(int(d['box'][0]) for d in dets)
        grupp_y1 = min(int(d['box'][1]) for d in dets)
        cv2.putText(frame, klassinfo['plural'], (grupp_x1, grupp_y1 - 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_SORTED, 2)

    return frame


print('Hjälpfunktioner definierade!')

## 4. Analysera en enskild bild

In [ ]:
# Ladda YOLO-modellen
try:
    modell = YOLO('best.pt')
    print('Modell laddad: best.pt')
except Exception:
    warnings.warn('best.pt hittades inte, använder yolov8n.pt som reserv.', UserWarning)
    modell = YOLO('yolov8n.pt')
    print('Modell laddad: yolov8n.pt (reserv)')

In [ ]:
# Ange sökväg till en testbild
BILD_SOKVAG = 'testbild.jpg'  # Ändra till din bildsökväg

bild = cv2.imread(BILD_SOKVAG)

if bild is None:
    print(f'Kunde inte ladda bilden: {BILD_SOKVAG}')
    print('Skapar en tom exempelbild...')
    bild = np.zeros((480, 640, 3), dtype=np.uint8)

# Kör YOLO-inferens
inferens = modell(bild, verbose=False)

# Extrahera detektioner
detektioner = []
for r in inferens:
    if r.boxes is None:
        continue
    for box in r.boxes:
        xyxy = box.xyxy[0].cpu().numpy()
        class_id = int(box.cls[0].cpu().numpy())
        conf = float(box.conf[0].cpu().numpy())
        if class_id in CLASS_NAMES:
            detektioner.append({'box': xyxy, 'class_id': class_id, 'conf': conf})

print(f'Antal detektioner: {len(detektioner)}')

# Bestäm sorteringsstatus
resultat = bestam_status(detektioner)

# Rita annotationer
annoterad = rita_pa_frame(bild.copy(), resultat)

# Visa med matplotlib (konvertera BGR → RGB)
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annoterad, cv2.COLOR_BGR2RGB))
plt.title('Avfallssorteringsanalys')
plt.axis('off')
plt.tight_layout()
plt.show()

# Skriv ut statussammanfattning
for r in resultat:
    klassnamn = CLASS_NAMES[r['class_id']]['singular']
    print(f"  {klassnamn} – Status: {r['status']} – Konfidensgrad: {r['conf']:.2f}")

## 5. Videoanalys

In [ ]:
# Videoanalys – webbkamera (0) eller videofil
# OBS: OpenCV-fönster kräver ett grafiskt gränssnitt.
# I en notebook-miljö utan skärm, spara frames som bilder istället.

VIDEO_KALLA = 0  # Sätt till 'video.mp4' för att analysera en fil

cap = cv2.VideoCapture(VIDEO_KALLA)

if not cap.isOpened():
    print(f'Kunde inte öppna videokällan: {VIDEO_KALLA}')
else:
    print('Videoström öppnad. Tryck "q" för att avsluta (i OpenCV-fönstret).')

    ram_nummer = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Videoströmmen är slut.')
            break

        # Kör analys
        inferens = modell(frame, verbose=False)
        detektioner = []
        for r in inferens:
            if r.boxes is None:
                continue
            for box in r.boxes:
                xyxy = box.xyxy[0].cpu().numpy()
                class_id = int(box.cls[0].cpu().numpy())
                conf = float(box.conf[0].cpu().numpy())
                if class_id in CLASS_NAMES:
                    detektioner.append({'box': xyxy, 'class_id': class_id, 'conf': conf})

        resultat = bestam_status(detektioner)
        annoterad = rita_pa_frame(frame.copy(), resultat)

        cv2.imshow('Avfallspolisen', annoterad)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        ram_nummer += 1

    cap.release()
    cv2.destroyAllWindows()
    print(f'Analys klar – {ram_nummer} frames bearbetades.')

## 6. Träning

In [ ]:
# Träna en ny YOLOv8-modell på avfallssorteringsdatasetet.
# Kräver att dataset.yaml och datamappen (dataset/) är korrekt uppsatta.

traning_modell = YOLO('yolov8n.pt')  # Starta från förtränad modell

traning_resultat = traning_modell.train(
    data='dataset.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name='waste_sorting_model',
    patience=20,
    save=True,
    plots=True,
)

print('Träning klar!')

## 7. Utvärdering

In [ ]:
# Ladda den tränade modellen (om träning kördes ovan)
try:
    utv_modell = YOLO('best.pt')
    print('Utvärderingsmodell laddad: best.pt')
except Exception:
    utv_modell = YOLO('yolov8n.pt')
    print('Utvärderingsmodell laddad: yolov8n.pt (reserv)')

# Kör validering mot test-datasetet
val_resultat = utv_modell.val(
    data='dataset.yaml',
    split='test',
    plots=True,
)

print(f'\nmAP50:    {val_resultat.box.map50:.4f}')
print(f'mAP50-95: {val_resultat.box.map:.4f}')

print('\nPer-klass mAP50:')
for i, klass_namn in enumerate(['Dryckeskartong', 'Konservburk', 'Pantburk']):
    if i < len(val_resultat.box.ap50):
        print(f'  {klass_namn}: {val_resultat.box.ap50[i]:.4f}')

print('\nConfusion Matrix och diagram sparades i runs/detect/val/')